In [5]:
import pygame
import sys
import random
import time

# Khởi tạo pygame
pygame.init()

# Màu sắc
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (200, 200, 200)

# Thiết lập màn hình
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Memory Game")

# Font chữ
font = pygame.font.SysFont(None, 36)
small_font = pygame.font.SysFont(None, 24)

class Circle:
    def __init__(self, x, y, radius, number):
        self.x = x
        self.y = y
        self.radius = radius
        self.number = number
        self.show_number = True
        self.clicked = False
        self.color = BLUE
    
    def draw(self):
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)
        if self.show_number:
            text = font.render(str(self.number), True, WHITE)
            text_rect = text.get_rect(center=(self.x, self.y))
            screen.blit(text, text_rect)
    
    def is_clicked(self, pos):
        distance = ((pos[0] - self.x) ** 2 + (pos[1] - self.y) ** 2) ** 0.5
        return distance <= self.radius

class Game:
    def __init__(self):
        self.level = 1
        self.circles = []
        self.show_numbers = True
        self.current_number = 1
        self.game_state = "ready"  # ready, memorize, playing, won, lost
        self.message = "Bắt đầu chơi!"
        self.start_time = 0
        self.level_complete_time = 0  # Thời điểm hoàn thành level
    
    def start_game(self):
        self.circles = []
        self.current_number = 1
        self.game_state = "memorize"
        self.message = "Ghi nhớ vị trí các số!"
        
        # Tạo n ô tròn với n = level + 2
        n = self.level + 2
        positions = []
        
        # Đảm bảo các ô tròn không chồng lên nhau
        radius = 40
        margin = 20
        max_attempts = 100
        
        for i in range(1, n + 1):
            attempts = 0
            while attempts < max_attempts:
                x = random.randint(radius + margin, WIDTH - radius - margin)
                y = random.randint(radius + margin, HEIGHT - 100 - radius - margin)
                
                # Kiểm tra xem vị trí có chồng lên vị trí đã có không
                valid_position = True
                for pos in positions:
                    distance = ((pos[0] - x) ** 2 + (pos[1] - y) ** 2) ** 0.5
                    if distance < 2 * radius + margin:
                        valid_position = False
                        break
                
                if valid_position:
                    positions.append((x, y))
                    self.circles.append(Circle(x, y, radius, i))
                    break
                
                attempts += 1
            
            # Nếu không tìm được vị trí hợp lệ sau nhiều lần thử, đặt vị trí mặc định
            if attempts >= max_attempts:
                x = random.randint(radius + margin, WIDTH - radius - margin)
                y = random.randint(radius + margin, HEIGHT - 100 - radius - margin)
                positions.append((x, y))
                self.circles.append(Circle(x, y, radius, i))
        
        self.show_numbers = True
        self.start_time = time.time()
    
    def hide_numbers(self):
        self.show_numbers = False
        for circle in self.circles:
            circle.show_number = False
        self.game_state = "playing"
        self.message = "Nhấn vào các ô theo thứ tự tăng dần!"
    
    def handle_click(self, pos):
        if self.game_state != "playing":
            return
        
        for circle in self.circles:
            if circle.is_clicked(pos) and not circle.clicked:
                if circle.number == self.current_number:
                    circle.clicked = True
                    circle.color = GREEN
                    circle.show_number = True
                    self.current_number += 1
                    
                    # Kiểm tra xem đã hoàn thành level chưa
                    if self.current_number > len(self.circles):
                        self.game_state = "won"
                        self.message = f"Chúc mừng! Bạn đã hoàn thành level {self.level}!"
                        self.level_complete_time = time.time()  # Lưu thời điểm hoàn thành
                else:
                    circle.color = RED
                    circle.show_number = True
                    self.game_state = "lost"
                    self.message = f"Game over! Bạn đã chọn sai ô. Bạn cần chọn ô số {self.current_number}."
                break
    
    def check_next_level(self):
        # Kiểm tra xem đã đến lúc chuyển level chưa
        if self.game_state == "won" and time.time() - self.level_complete_time >= 0.5:
            self.level += 1
            self.start_game()

def main():
    clock = pygame.time.Clock()
    game = Game()
    
    running = True
    while running:
        screen.fill(WHITE)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if game.game_state == "ready":
                    game.start_game()
                elif game.game_state == "playing":
                    game.handle_click(event.pos)
                elif game.game_state == "lost":
                    game.level = 1
                    game.game_state = "ready"
        
        # Ẩn số sau 1 giây
        if game.game_state == "memorize" and time.time() - game.start_time > 1:
            game.hide_numbers()
        
        # Kiểm tra chuyển level sau 0.5 giây khi hoàn thành level
        if game.game_state == "won":
            game.check_next_level()
        
        # Vẽ các ô tròn
        for circle in game.circles:
            circle.draw()
        
        # Hiển thị thông tin
        level_text = small_font.render(f"Level: {game.level}", True, BLACK)
        screen.blit(level_text, (10, HEIGHT - 80))
        
        message_text = font.render(game.message, True, BLACK)
        message_rect = message_text.get_rect(center=(WIDTH // 2, HEIGHT - 50))
        screen.blit(message_text, message_rect)
        
        # Hiển thị nút "Start" khi ở trạng thái sẵn sàng
        if game.game_state == "ready":
            pygame.draw.rect(screen, BLUE, (WIDTH // 2 - 60, HEIGHT // 2 - 30, 120, 60))
            start_text = font.render("Start", True, WHITE)
            start_rect = start_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(start_text, start_rect)
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

SystemExit: 